# c2_LPF_float

In [38]:
import numpy as np
import pandas as pd
import os
from scipy.signal import butter, lfilter, freqz

In [39]:
def butter_lowpass(cutoff, fs, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=4):
    b, a = butter_lowpass(cutoff, fs, order=order)
    x = np.trunc(np.array(data, dtype=np.float64)).astype(np.int64)
    y = lfilter(b, a, x, axis=-1, zi=None)
    return y

# Setting standard filter requirements.
order = 4
fs = 200
cutoff = 5

b, a = butter_lowpass(cutoff, fs, order)

# Creating the data for filteration
T = 200         # value taken in seconds
n = int(T * fs) # indicates total samples
t = np.linspace(0, T, n, endpoint=False)

# ADL path

In [40]:
paths = []

for i in range(1, 20):
    root, file = os.path.split("../../Dataset/SA01/D01_SA01_R01.csv")
    if i < 10:
        file = file[:2] + str(i) + file[3:]
        tmp = os.path.join(root, file)
        paths.append(tmp)
        if i > 4:
            for j in range(2, 6):
                file_new = file[:-5] + str(j) + file[-4:]
                tmp = os.path.join(root, file_new)
                paths.append(tmp)
    if i >= 10:
        file = file[:1] + str(i) + file[3:]
        for j in range(1, 6):
            file = file[:-5] + str(j) + file[-4:]
            tmp = os.path.join(root, file)
            paths.append(tmp)

paths[4:14] = [ '../../Dataset/SA01/D05_SA01_R01_1.csv',
                '../../Dataset/SA01/D05_SA01_R01_2.csv',
                '../../Dataset/SA01/D05_SA01_R02_1.csv',
                '../../Dataset/SA01/D05_SA01_R02_2.csv',
                '../../Dataset/SA01/D05_SA01_R03_1.csv',
                '../../Dataset/SA01/D05_SA01_R03_2.csv',
                '../../Dataset/SA01/D05_SA01_R04_1.csv',
                '../../Dataset/SA01/D05_SA01_R04_2.csv',
                '../../Dataset/SA01/D05_SA01_R05_1.csv',
                '../../Dataset/SA01/D05_SA01_R05_2.csv',
                '../../Dataset/SA01/D06_SA01_R01_1.csv',
                '../../Dataset/SA01/D06_SA01_R01_2.csv',
                '../../Dataset/SA01/D06_SA01_R02_1.csv',
                '../../Dataset/SA01/D06_SA01_R02_2.csv',
                '../../Dataset/SA01/D06_SA01_R03_1.csv',
                '../../Dataset/SA01/D06_SA01_R03_2.csv',
                '../../Dataset/SA01/D06_SA01_R04_1.csv',
                '../../Dataset/SA01/D06_SA01_R04_2.csv',
                '../../Dataset/SA01/D06_SA01_R05_1.csv',
                '../../Dataset/SA01/D06_SA01_R05_2.csv']

paths[-15:-10] =['../../Dataset/SA01/D17_SA01_R01_1.csv',
                 '../../Dataset/SA01/D17_SA01_R01_2.csv',
                 '../../Dataset/SA01/D17_SA01_R02_1.csv',
                 '../../Dataset/SA01/D17_SA01_R02_2.csv',
                 '../../Dataset/SA01/D17_SA01_R03_1.csv',
                 '../../Dataset/SA01/D17_SA01_R03_2.csv',
                 '../../Dataset/SA01/D17_SA01_R04_1.csv',
                 '../../Dataset/SA01/D17_SA01_R04_2.csv',
                 '../../Dataset/SA01/D17_SA01_R05_1.csv',
                 '../../Dataset/SA01/D17_SA01_R05_2.csv']


subject_list = ['SA01', 'SA02', 'SA03', 'SA04', 'SA05',
                'SA06', 'SA07', 'SA08', 'SA09', 'SA10',
                'SA11', 'SA12', 'SA13', 'SA14', 'SA15',
                'SA16', 'SA17', 'SA18', 'SA19', 'SA20',
                'SA21', 'SA22', 'SA23',
                'SE01', 'SE02', 'SE03', 'SE04', 'SE05',
                'SE06', 'SE07', 'SE08', 'SE09', 'SE10',
                'SE11', 'SE12', 'SE13', 'SE14', 'SE15']


total_adl_path = []

for path in paths:
    for subject_name in subject_list:
        path_rename = path.replace('SA01', subject_name)
        total_adl_path.append(path_rename)

len(total_adl_path)

3572

# ADL data 전처리

In [41]:
adl_list = []

for path in total_adl_path:
    try:
        data = pd.read_csv(path, header=None)
        data = data.iloc[:, :3]
        data = np.abs(data)
                    
        max_value1 = data[0][200:].max()
        max_value3 = data[2][200:].max()
        
        max_idx1 = data[ data[0] == max_value1 ].index[0]
        max_idx3 = data[ data[2] == max_value3 ].index[0]
        
        min_idx = min(max_idx1, max_idx3)
        sliced_df = data[min_idx - 200 : min_idx - 40]
        
        col_1_lpf = butter_lowpass_filter(sliced_df[0].to_numpy(), cutoff, fs, order).astype('float64')
        col_3_lpf = butter_lowpass_filter(sliced_df[2].to_numpy(), cutoff, fs, order).astype('float64')

        col_1 = col_1_lpf*col_1_lpf
        col_3 = col_3_lpf*col_3_lpf
        
        C_nine = np.sqrt(col_1 + col_3).tolist()
        adl_list.append(C_nine)
        # adl_first = pd.concat([list_C_nine, adl_first], axis=0, ignore_index=True)
         
    except:
        pass
    

adl_df = pd.DataFrame(adl_list)
print(adl_df.shape)
row, col = adl_df.shape


target_col = row * [0]
adl_df['target'] = target_col

print(adl_df.shape)
adl_df

(3209, 160)
(3209, 161)


,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,target
0,0.001902,0.016392,0.070792,0.209120,0.484566,0.950771,1.654261,2.628056,3.887504,5.429040,...,12.345714,11.674087,11.399092,11.256729,11.056370,10.698986,10.172520,9.549263,8.989232,0
1,0.001816,0.014713,0.059451,0.164328,0.357672,0.663298,1.099225,1.677603,2.403344,3.271704,...,21.239457,20.734987,20.342576,20.117295,20.104025,20.331127,20.807075,21.521362,22.448932,0
2,0.005000,0.042328,0.179048,0.516954,1.169317,2.239287,3.806907,5.922823,8.606599,11.847755,...,112.699696,112.415849,111.922427,111.217917,110.302797,109.179324,107.851850,106.327333,104.615599,0
3,0.002714,0.023321,0.100369,0.295374,0.681615,1.332021,2.310642,3.667386,5.435141,7.628496,...,90.775029,88.743205,86.608922,84.416787,82.213344,80.045589,77.959660,75.999388,74.204980,0
4,0.001485,0.012718,0.054622,0.160773,0.372144,0.731500,1.278873,2.047723,3.062195,4.335583,...,25.233263,26.335399,27.357993,28.282809,29.089051,29.754654,30.261145,30.599177,30.771215,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3204,0.000978,0.008477,0.036682,0.108320,0.250534,0.490523,0.852841,1.358227,2.022947,2.857790,...,32.352092,33.277918,35.056976,37.732682,41.319070,45.796445,51.111043,57.178408,63.890407,0
3205,0.002680,0.023080,0.099758,0.295520,0.687969,1.358319,2.382068,3.822044,5.724175,8.115373,...,104.448345,104.914718,105.462122,106.099564,106.833418,107.667124,108.601197,109.633082,110.756019,0
3206,0.002468,0.021293,0.091987,0.271767,0.630092,1.238389,2.162512,3.457812,5.166435,7.315829,...,68.340085,61.998688,58.001687,56.176917,56.271126,57.994053,61.051026,65.154117,70.023585,0
3207,0.000279,0.002420,0.010570,0.031777,0.075130,0.150394,0.267035,0.433829,0.658457,0.946871,...,79.188468,72.846084,66.474693,60.282143,54.503604,49.392384,45.196820,42.117847,40.252163,0


# ADL data 저장

In [42]:
final = pd.DataFrame(adl_df)
final.to_csv('../result/a_c2_LPF_float.csv', index=False)

# Fall path

In [43]:
fall_list = []

for i in range(1, 16):
    root, file = os.path.split("../../Dataset/SA01/F01_SA01_R01.csv")
    if i < 10:
        file = file[:2] + str(i) + file[3:]
        tmp = os.path.join(root, file)
        for j in range(1, 6):
            file_new = file[:-5] + str(j) + file[-4:]
            tmp = os.path.join(root, file_new)
            fall_list.append(tmp)
    if i >= 10:
        file = file[:1] + str(i) + file[3:]
        tmp = os.path.join(root, file)
        for j in range(1, 6):
            file_new = file[:-5] + str(j) + file[-4:]
            tmp = os.path.join(root, file_new)
            fall_list.append(tmp)
            
print(fall_list)

subject_list = ['SA01', 'SA02', 'SA03', 'SA04', 'SA05',
                'SA06', 'SA07', 'SA08', 'SA09', 'SA10',
                'SA11', 'SA12', 'SA13', 'SA14', 'SA15',
                'SA16', 'SA17', 'SA18', 'SA19', 'SA20',
                'SA21', 'SA22', 'SA23', 'SE06']

total_fall_path = []

for path in fall_list:
    for subject_name in subject_list:
        path_rename = path.replace('SA01', subject_name)
        total_fall_path.append(path_rename)

total_fall_path

['../../Dataset/SA01/F01_SA01_R01.csv', '../../Dataset/SA01/F01_SA01_R02.csv', '../../Dataset/SA01/F01_SA01_R03.csv', '../../Dataset/SA01/F01_SA01_R04.csv', '../../Dataset/SA01/F01_SA01_R05.csv', '../../Dataset/SA01/F02_SA01_R01.csv', '../../Dataset/SA01/F02_SA01_R02.csv', '../../Dataset/SA01/F02_SA01_R03.csv', '../../Dataset/SA01/F02_SA01_R04.csv', '../../Dataset/SA01/F02_SA01_R05.csv', '../../Dataset/SA01/F03_SA01_R01.csv', '../../Dataset/SA01/F03_SA01_R02.csv', '../../Dataset/SA01/F03_SA01_R03.csv', '../../Dataset/SA01/F03_SA01_R04.csv', '../../Dataset/SA01/F03_SA01_R05.csv', '../../Dataset/SA01/F04_SA01_R01.csv', '../../Dataset/SA01/F04_SA01_R02.csv', '../../Dataset/SA01/F04_SA01_R03.csv', '../../Dataset/SA01/F04_SA01_R04.csv', '../../Dataset/SA01/F04_SA01_R05.csv', '../../Dataset/SA01/F05_SA01_R01.csv', '../../Dataset/SA01/F05_SA01_R02.csv', '../../Dataset/SA01/F05_SA01_R03.csv', '../../Dataset/SA01/F05_SA01_R04.csv', '../../Dataset/SA01/F05_SA01_R05.csv', '../../Dataset/SA01/F06_

['../../Dataset/SA01/F01_SA01_R01.csv',
 '../../Dataset/SA02/F01_SA02_R01.csv',
 '../../Dataset/SA03/F01_SA03_R01.csv',
 '../../Dataset/SA04/F01_SA04_R01.csv',
 '../../Dataset/SA05/F01_SA05_R01.csv',
 '../../Dataset/SA06/F01_SA06_R01.csv',
 '../../Dataset/SA07/F01_SA07_R01.csv',
 '../../Dataset/SA08/F01_SA08_R01.csv',
 '../../Dataset/SA09/F01_SA09_R01.csv',
 '../../Dataset/SA10/F01_SA10_R01.csv',
 '../../Dataset/SA11/F01_SA11_R01.csv',
 '../../Dataset/SA12/F01_SA12_R01.csv',
 '../../Dataset/SA13/F01_SA13_R01.csv',
 '../../Dataset/SA14/F01_SA14_R01.csv',
 '../../Dataset/SA15/F01_SA15_R01.csv',
 '../../Dataset/SA16/F01_SA16_R01.csv',
 '../../Dataset/SA17/F01_SA17_R01.csv',
 '../../Dataset/SA18/F01_SA18_R01.csv',
 '../../Dataset/SA19/F01_SA19_R01.csv',
 '../../Dataset/SA20/F01_SA20_R01.csv',
 '../../Dataset/SA21/F01_SA21_R01.csv',
 '../../Dataset/SA22/F01_SA22_R01.csv',
 '../../Dataset/SA23/F01_SA23_R01.csv',
 '../../Dataset/SE06/F01_SE06_R01.csv',
 '../../Dataset/SA01/F01_SA01_R02.csv',


# Fall 데이터 전처리

In [44]:
fall_list = []

for path in total_fall_path:
    try:
        data = pd.read_csv(path, header=None)
        data = data.iloc[:, :3]
        data = np.abs(data)
                    
        max_value1 = data[0][1000:].max()
        max_value3 = data[2][1000:].max()
        
        max_idx1 = data[ data[0] == max_value1 ].index[0]
        max_idx3 = data[ data[2] == max_value3 ].index[0]
        
        min_idx = min(max_idx1, max_idx3)
        sliced_df = data[min_idx - 200 : min_idx - 40]
        
        col_1_lpf = butter_lowpass_filter(sliced_df[0].to_numpy(), cutoff, fs, order).astype('float64')
        col_3_lpf = butter_lowpass_filter(sliced_df[2].to_numpy(), cutoff, fs, order).astype('float64')

        col_1 = col_1_lpf*col_1_lpf
        col_3 = col_3_lpf*col_3_lpf
        
        C_nine = np.sqrt(col_1 + col_3).tolist()
        fall_list.append(C_nine)
        # adl_first = pd.concat([list_C_nine, adl_first], axis=0, ignore_index=True)
         
    except:
        pass
    

fall_df = pd.DataFrame(fall_list)
print(fall_df.shape)
row, col = fall_df.shape


target_col = row * [1]
fall_df['target'] = target_col

print(fall_df.shape)
fall_df

(1799, 160)
(1799, 161)


,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,target
0,0.000312,0.002842,0.012662,0.037755,0.086900,0.167778,0.285853,0.443604,0.640368,0.872794,...,102.522495,106.766483,112.842963,120.539949,129.572837,139.598153,150.232478,161.074282,171.726749,1
1,0.001414,0.012243,0.053448,0.160532,0.379551,0.760824,1.352309,2.193876,3.313600,4.726304,...,79.439662,76.908814,74.268701,71.557879,68.830068,66.153687,63.608990,61.283281,59.264808,1
2,0.003155,0.027072,0.116427,0.342673,0.791590,1.549895,2.695890,4.293504,6.388306,9.005025,...,157.581673,161.741803,165.524294,168.972506,172.145941,175.110940,177.932279,180.666473,183.357470,1
3,0.003429,0.029308,0.125611,0.368806,0.850752,1.664324,2.892750,4.603003,6.842048,9.634782,...,70.879304,69.319257,68.379630,68.121503,68.596468,69.855979,71.973414,75.074711,79.361611,1
4,0.004935,0.042195,0.180240,0.525293,1.198626,2.313989,3.962668,6.204686,9.064930,12.533749,...,133.225113,141.719774,149.988436,157.847722,165.106846,171.581201,177.111143,181.581252,184.937593,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1794,0.003032,0.026046,0.112254,0.331540,0.769692,1.516322,2.655449,4.258896,6.381991,9.060675,...,165.454409,165.934192,166.482435,167.091517,167.748101,168.434731,169.131300,169.816201,170.467224,1
1795,0.002875,0.024625,0.105677,0.310506,0.716898,1.404803,2.448272,3.909694,5.835553,8.253826,...,236.742387,237.433696,237.601779,237.343131,236.765922,235.985907,235.124623,234.306353,233.651481,1
1796,0.003076,0.026348,0.113177,0.333059,0.770361,1.512245,2.639454,4.219499,6.302257,8.917711,...,146.928397,144.747611,142.739397,140.991685,139.595853,138.644275,138.226830,138.426367,139.313180,1
1797,0.002389,0.020524,0.088436,0.260871,0.604026,1.185517,2.067127,3.299685,4.919751,6.947947,...,117.884275,119.692180,121.391945,122.950518,124.322690,125.451716,126.271536,126.711314,126.701199,1


# Fall data 저장

In [45]:
final = pd.DataFrame(fall_df)
final.to_csv('../result/f_c2_LPF_float.csv', index=False)

# All data 저장

In [46]:
adl = pd.read_csv('../result/a_c2_LPF_float.csv')
fall = pd.read_csv('../result/f_c2_LPF_float.csv')

fall_data = pd.DataFrame(fall)
print(fall_data.shape)
adl_data = pd.DataFrame(adl)
print(adl_data.shape)

file_list = [fall_data, adl_data]
result = pd.concat(file_list)
result.to_csv('../result/c2_LPF_float.csv', index=False)
print(result.shape)
result

(1799, 161)
(3209, 161)
(5008, 161)


,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,target
0,0.000312,0.002842,0.012662,0.037755,0.086900,0.167778,0.285853,0.443604,0.640368,0.872794,...,102.522495,106.766483,112.842963,120.539949,129.572837,139.598153,150.232478,161.074282,171.726749,1
1,0.001414,0.012243,0.053448,0.160532,0.379551,0.760824,1.352309,2.193876,3.313600,4.726304,...,79.439662,76.908814,74.268701,71.557879,68.830068,66.153687,63.608990,61.283281,59.264808,1
2,0.003155,0.027072,0.116427,0.342673,0.791590,1.549895,2.695890,4.293504,6.388306,9.005025,...,157.581673,161.741803,165.524294,168.972506,172.145941,175.110940,177.932279,180.666473,183.357470,1
3,0.003429,0.029308,0.125611,0.368806,0.850752,1.664324,2.892750,4.603003,6.842048,9.634782,...,70.879304,69.319257,68.379630,68.121503,68.596468,69.855979,71.973414,75.074711,79.361611,1
4,0.004935,0.042195,0.180240,0.525293,1.198626,2.313989,3.962668,6.204686,9.064930,12.533749,...,133.225113,141.719774,149.988436,157.847722,165.106846,171.581201,177.111143,181.581252,184.937593,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3204,0.000978,0.008477,0.036682,0.108320,0.250534,0.490523,0.852841,1.358227,2.022947,2.857790,...,32.352092,33.277918,35.056976,37.732682,41.319070,45.796445,51.111043,57.178408,63.890407,0
3205,0.002680,0.023080,0.099758,0.295520,0.687969,1.358319,2.382068,3.822044,5.724175,8.115373,...,104.448345,104.914718,105.462122,106.099564,106.833418,107.667124,108.601197,109.633082,110.756019,0
3206,0.002468,0.021293,0.091987,0.271767,0.630092,1.238389,2.162512,3.457812,5.166435,7.315829,...,68.340085,61.998688,58.001687,56.176917,56.271126,57.994053,61.051026,65.154117,70.023585,0
3207,0.000279,0.002420,0.010570,0.031777,0.075130,0.150394,0.267035,0.433829,0.658457,0.946871,...,79.188468,72.846084,66.474693,60.282143,54.503604,49.392384,45.196820,42.117847,40.252163,0
